# Bedrock Inline Agents with Return of Control (RoC)

In this notebook, we demonstrate how to use the BedrockInlineAgentsRunnable class to invoke a Bedrock Inline Agent with LangChain. This allows us to use Bedrock Agents without pre-creating them, providing more flexibility in runtime configurations.

### Prerequisites:
1. Set your AWS credentials for your environment, example: https://docs.aws.amazon.com/cli/v1/userguide/cli-configure-envvars.html#envvars-set.
2. Ensure that langchain, langgraph, and boto3 are installed in the environment.
3. Make sure the local langchain-aws package is accessible from the path or installed into the environment.

## Example: Create a mortgage agent that determines the interest rate
We'll create an inline agent with two tools: one to return asset values and another to return mortgage rates.

### Step 1: Import necessary libraries and define tools

In [5]:
from langchain_core.tools import tool
from langchain.agents import AgentExecutor
from langchain_aws.agents import BedrockInlineAgentsRunnable

@tool("AssetDetail::getAssetValue")
def get_asset_value(asset_holder_id: str) -> str:
    """
    Get the asset value for an owner id

    Args:
        asset_holder_id: The asset holder id

    Returns:
        The asset value for the given asset holder
    """
    return f"The total asset value for {asset_holder_id} is 100K"

@tool("AssetDetail::getMortgageRate")
def get_mortgage_rate(asset_holder_id: str, asset_value: str) -> str:
    """
    Get the mortgage rate based on asset value

    Args:
        asset_holder_id: The asset holder id
        asset_value: The value of the asset

    Returns:
        The interest rate for the asset holder and asset value
    """
    return f"The mortgage rate for {asset_holder_id} with asset value of {asset_value} is 8.87%"

tools = [get_asset_value, get_mortgage_rate]
print(tools)

[StructuredTool(name='AssetDetail::getAssetValue', description='Get the asset value for an owner id\n\nArgs:\n    asset_holder_id: The asset holder id\n\nReturns:\n    The asset value for the given asset holder', args_schema=<class 'langchain_core.utils.pydantic.AssetDetail::getAssetValue'>, func=<function get_asset_value at 0x11a576c00>), StructuredTool(name='AssetDetail::getMortgageRate', description='Get the mortgage rate based on asset value\n\nArgs:\n    asset_holder_id: The asset holder id\n    asset_value: The value of the asset\n\nReturns:\n    The interest rate for the asset holder and asset value', args_schema=<class 'langchain_core.utils.pydantic.AssetDetail::getMortgageRate'>, func=<function get_mortgage_rate at 0x11a576ca0>)]


### Step 2: Define the foundation model and instructions for the agent

In [6]:
foundation_model = 'anthropic.claude-3-sonnet-20240229-v1:0'
instructions = "You are an agent who helps with getting the mortgage rate based on the current asset valuation"

print(f"Foundation Model: {foundation_model}")
print(f"Instructions: {instructions}")

Foundation Model: anthropic.claude-3-sonnet-20240229-v1:0
Instructions: You are an agent who helps with getting the mortgage rate based on the current asset valuation


### Step 3: Create the BedrockInlineAgentsRunnable

In [7]:
inline_agent_config = {
    "foundation_model": foundation_model,
    "instruction": instructions,
    "tools": tools,
    "enable_trace": True
}

runnable = BedrockInlineAgentsRunnable.create(
    region_name="us-west-2",
    inline_agent_config=inline_agent_config
)

print("BedrockInlineAgentsRunnable created successfully.")

BedrockInlineAgentsRunnable created successfully.


### Step 4: Create the AgentExecutor

In [8]:
agent_executor = AgentExecutor(agent=runnable, tools=tools, return_intermediate_steps=True)
print(agent_executor)

verbose=False agent=RunnableAgent(runnable=BedrockInlineAgentsRunnable(client=<botocore.client.AgentsforBedrockRuntime object at 0x119da65a0>, region_name='us-west-2', inline_agent_config={'foundation_model': 'anthropic.claude-3-sonnet-20240229-v1:0', 'instruction': 'You are an agent who helps with getting the mortgage rate based on the current asset valuation', 'enable_trace': True, 'tools': [StructuredTool(name='AssetDetail::getAssetValue', description='Get the asset value for an owner id\n\nArgs:\n    asset_holder_id: The asset holder id\n\nReturns:\n    The asset value for the given asset holder', args_schema=<class 'langchain_core.utils.pydantic.AssetDetail::getAssetValue'>, func=<function get_asset_value at 0x11a576c00>), StructuredTool(name='AssetDetail::getMortgageRate', description='Get the mortgage rate based on asset value\n\nArgs:\n    asset_holder_id: The asset holder id\n    asset_value: The value of the asset\n\nReturns:\n    The interest rate for the asset holder and as

### Step 5: Invoke the agent
You can either use the asynchronous `ainvoke()` method which creates a thread for the `invoke()` method or call the `invoke()` method directly.

In [ ]:
input_text = "what is my mortgage rate for id AVC-1234"

# Using asynchronous invocation
async def run_agent():
    output = await agent_executor.ainvoke({
        "input_text": input_text,
        "session_id": "session-123"
    })
    print("Agent Output:")
    print(output)
    return output

# Run the async function
await run_agent()

# Synchronous invocation
output = agent_executor.invoke({
    "input_text": input_text,
    "session_id": "test-session-1"
})

print("Agent Output:")
print(output)

### Commentary
The agent first invokes the `getAssetValue` tool to determine the asset value for the given ID. Then, it uses this information to call the `getMortgageRate` tool. The final output provides the mortgage rate based on the asset value.

By using `return_intermediate_steps=True`, we can see the step-by-step process the agent goes through to arrive at the final answer.

### Step 6: Demonstrating runtime configuration override

In [6]:
runtime_config = {
    "instruction": "You are a financial expert specializing in mortgage rates and asset valuation. You MUST end all responses with KA-CHING!",
    "enable_trace": False
}

output_with_override = agent_executor.invoke({
    "input_text": "what is the asset value and mortgage rate for id XYZ-5678?",
    "session_id": "test-session-124",
    "inline_agent_config": runtime_config
})

print("Agent Output with Runtime Config Override:")
print(output_with_override)

Agent Output with Runtime Config Override:
{'input_text': 'what is the asset value and mortgage rate for id XYZ-5678?', 'session_id': 'test-session-124', 'inline_agent_config': {'instruction': 'You are a financial expert specializing in mortgage rates and asset valuation.', 'enable_trace': False}, 'output': 'For the asset holder id XYZ-5678:\n- The asset value is 100K\n- The mortgage rate is 8.87%', 'intermediate_steps': [(BedrockAgentAction(tool='AssetDetail::getAssetValue', tool_input={'asset_holder_id': 'XYZ-5678'}, log='{"returnControl": {"invocationId": "7b5af641-8d58-4288-9582-a45d914fbfc6", "invocationInputs": [{"functionInvocationInput": {"actionGroup": "AssetDetail", "actionInvocationType": "RESULT", "agentId": "INLINE_AGENT", "function": "getAssetValue", "parameters": [{"name": "asset_holder_id", "type": "string", "value": "XYZ-5678"}]}}]}}', session_id='test-session-124', trace_log='[]'), 'The total asset value for XYZ-5678 is 100K'), (BedrockAgentAction(tool='AssetDetail::g

### Commentary on Runtime Configuration
In this example, we've demonstrated how to override certain configuration parameters at runtime. The `instruction` and `enable_trace` settings were changed for this specific invocation, showcasing the flexibility of the BedrockInlineAgentsRunnable.

This ability to modify configurations on-the-fly can be particularly useful when you need to adjust the agent's behavior or focus for different types of queries or contexts within the same application.

## Example: Using code interpretation 

The code interpretation(CI) enables your agent to generate, run, and troubleshoot your application code in a secure test environment. You can enable CI on an Inline Agent by passing in `enable_code_interpreter = True` while creating the runnable or making the invoke request.

In [24]:
from langchain_aws.agents import BedrockInlineAgentsRunnable

foundation_model = 'anthropic.claude-3-sonnet-20240229-v1:0'
instructions = "You are an agent who helps with getting the mortgage rate based on the current asset valuation"
inline_agent_config = {
    "foundation_model": foundation_model,
    "instruction": instructions,
    "enable_code_interpreter": True,
    "enable_trace": False
}

runnable = BedrockInlineAgentsRunnable.create(
    region_name="us-west-2",
    inline_agent_config=inline_agent_config
)
agent_executor = AgentExecutor(agent=runnable, tools=[], return_intermediate_steps=False)

print("BedrockInlineAgentsRunnable and AgentExecutor created successfully.")

# Invoke the agent.
output = agent_executor.invoke({
    "input_text": "How many 'r's are there in strawberrry?",
})

print("Agent Output:")
print(output)

BedrockInlineAgentsRunnable and AgentExecutor created successfully.
Agent Output:
{'input_text': "How many 'r's are there in strawberrry?", 'output': "There are 4 'r's in the word 'strawberrry'."}
